In [36]:
import numpy as np
import random
import math
from math import exp

In [37]:
sudokuPuzzle = np.array([[0,0,0,0,3,0,6,0,0],
                         [5,0,0,9,0,0,4,0,0],
                         [0,8,0,6,0,7,0,0,9],
                         [0,7,0,0,0,0,8,0,1],
                         [0,5,0,0,8,0,0,2,0],
                         [3,0,8,0,0,0,0,5,0],
                         [1,0,0,8,0,4,0,9,0],
                         [0,0,2,0,0,6,0,0,5],
                         [0,0,9,0,1,0,0,0,0]])
#sudokuPuzzle

In [38]:
def copy_grid(sudoku):
    copiedPuzzle = []
    for i in range(len(sudoku)):
        copiedPuzzle.append(list(sudoku[i]))
    return copiedPuzzle
#copy_grid(sudokuPuzzle)

In [39]:
def square_coordinates(sudoku, r, c):
    square = []
    row = []
    square_size = int(math.sqrt(len(sudoku)))
    for i in range((r - 1) * square_size, square_size + (r - 1) * square_size  ):
        for j in range((c - 1) * square_size, square_size + (c - 1) * square_size ):
            row.append(sudoku[i][j])
        square.append(row)
        row = []
    return square
            
    
#square_coordinates(sudokuPuzzle, 3, 3)
    

In [40]:
def nonFixedPositions(sudoku, r, c):
    emptyIndices = []
    square_size = int(math.sqrt(len(sudoku)))
    square = square_coordinates(sudoku, r, c)
    for i in range(square_size):
        for j in range(square_size):
            if(square[i][j] == 0):
                emptyIndices.append([i, j])
    return emptyIndices
#nonFixedPositions(sudokuPuzzle, 1, 2)

In [41]:
def pickRandomUnfixedPosition(sudoku, i, j):
    randomIndex = random.randint(0, len(nonFixedPositions(sudoku, i, j)) - 1)
    return nonFixedPositions(sudoku, i, j)[randomIndex]
pickRandomUnfixedPosition(sudokuPuzzle, 1, 2)

[1, 2]

In [42]:
def possibleNumsList(sudoku, i, j):
    nonPossibilities = []
    possibilities = []
    square = square_coordinates(sudoku, i, j)
    for row in square:
        for num in row:
            if (num != 0):
                nonPossibilities.append(num)
    for n in range(1, len(sudoku) + 1):
        if (n not in nonPossibilities):
            possibilities.append(n)
    return possibilities

#possibleNumsList(sudokuPuzzle, 2, 2)

In [43]:
def fillBox(sudoku, i, j):
    box = square_coordinates(sudoku, i, j)
    listOfPossibilities = possibleNumsList(sudokuPuzzle, i, j)
    notFixedPositions = nonFixedPositions(sudoku, i, j)
    for x in range(len(notFixedPositions)):
       box[notFixedPositions[x][0]][notFixedPositions[x][1]] = listOfPossibilities[x]
    return np.array(box)
#fillBox(sudokuPuzzle, 1, 2)

In [44]:
def fillAllBoxes(sudoku):
    A = []
    B = []
    square_size = int(math.sqrt(len(sudoku)))
    for i in range(1, square_size + 1):
        for j in range(1, square_size + 1):
            A.append(fillBox(sudoku, i, j))

###copied this part completely cause nahi ho raha tha, we need to 
###figure a better way to do this.
    for i in range(3):    
        B.append(np.concatenate((A[0][i], A[1][i], A[2][i]), axis=None).tolist())
    for i in range(3):    
        B.append(np.concatenate((A[3][i], A[4][i], A[5][i]), axis=None).tolist())
    for i in range(3):    
        B.append(np.concatenate((A[6][i], A[7][i], A[8][i]), axis=None).tolist())
    return B
fillAllBoxes(sudokuPuzzle)

[[1, 2, 3, 1, 3, 2, 6, 1, 2],
 [5, 4, 6, 9, 4, 5, 4, 3, 5],
 [7, 8, 9, 6, 8, 7, 7, 8, 9],
 [1, 7, 2, 1, 2, 3, 8, 3, 1],
 [4, 5, 6, 4, 8, 5, 4, 2, 6],
 [3, 9, 8, 6, 7, 9, 7, 5, 9],
 [1, 3, 4, 8, 2, 4, 1, 9, 2],
 [5, 6, 2, 3, 5, 6, 3, 4, 5],
 [7, 8, 9, 7, 1, 9, 6, 7, 8]]

In [45]:
filledSudoku = fillAllBoxes(sudokuPuzzle)
filledSudokuCopy = copy_grid(filledSudoku)
#filledSudoku

In [46]:
##Cost is the numbers that are missing from the line
"""for example in line 1 [1, 2, 3, 1, 3, 2, 6, 1, 2]
numbers 4, 5, 7, 8, 9 are missing, so cost = 5

"equate cost to the number of
constraint violations"
"""
def costOfRow(sudoku):
    costList = []
    for i in range(len(sudoku)):
        row = sudoku[i]
        cost = 0
        for num in range(len(row)):
            if row.count(num + 1) == 0:
                cost += 1
        costList.append(cost)
    return sum(costList)
#costOfRow(filledSudoku)

In [47]:
def costOfColumn(sudoku):
    costList = []
    transposedSudoku = np.transpose(sudoku)
    for i in range(len(sudoku)):
        col = transposedSudoku[i]
        cost = 0
        for num in range(len(col)):
            if col.tolist().count(num + 1) == 0:
                cost += 1
        costList.append(cost)
    return sum(costList)
#costOfColumn(filledSudoku)

In [48]:
def totalCost(sudoku):
    return costOfRow(sudoku) + costOfColumn(sudoku)